In [2]:
'''
The following notebook is an example of automating the iterative generation of input .txt files for a specialized NMR simulation of CdTe in a program called "SIMPSON."
It itself requires the input of a .txt file that lists atomic coordinates of atomic lattice sites. Here, it is "CdTe_mp-405_24.txt," and is included in the folder.

SIMPSON requires certain inputs:
    atomic coordinates 
    coupling strengths
and Simpson outputs:
    Simulated NMR spectra and lineshapes

The issue with CdTe is that only 10% of Cd and 10% of Te isotopes are NMR-relevant, and they are randomly distributed.  This specific notebook takes the input atomic 
coordinates of the 24 closest atomic sites around a Cd-113 site, and assigns 2 of the 12 possible Te sites to host Te-125 atoms, and 3 of the possible Cd sites to host
Cd-113 atoms. This is performed iteratively, until all possible configurations are sampled. There is an input file for each configuration generated. Each configuration's
distances and coupling strengths are calculated on the fly.
'''

'\nThe following notebook is an example of automating the iterative generation of input .txt files for a specialized NMR simulation of CdTe in a program called "SIMPSON."\nIt itself requires the input of a .txt file that lists atomic coordinates of atomic lattice sites. Here, it is "CdTe_mp-405_24.txt"\n\nSIMPSON requires certain inputs:\n    atomic coordinates \n    coupling strengths\nand Simpson outputs:\n    Simulated NMR spectra and lineshapes\n\nThe issue with CdTe is that only 10% of Cd and 10% of Te isotopes are NMR-relevant, and they are randomly distributed.  This specific notebook takes the input atomic \ncoordinates of the 24 closest atomic sites around a Cd-113 site, and assigns 2 of the 12 possible Te sites to host Te-125 atoms, and 3 of the possible Cd sites to host\nCd-113 atoms. This is performed iteratively, until all possible configurations are sampled. There is an input file for each configuration generated. Each configuration\'s\ndistances and coupling strengths ar

In [3]:
#1 Te spin in neighborhood of an isolated Cd
import numpy as np
import math


#Assorted gyromagnetic ratios (constants) needed for definitions
g111 = -56.93*10**6/2/np.pi    #Gyromagnetic ratio of Cd-111, Hz/Tesla 
g113 = -59.55*10**6/2/np.pi    #Gyromagnetic ratio of Cd-113, Hz/Tesla 
g125 = -84.98*10**6/2/np.pi    #Gyromagnetic ratio of Te-125, Hz/Tesla 
hbar = 1.0545718*10**-34       #Reduced Planck constant, Joule-seconds

def bng(atom1,atom2):
    '''
    Calculates the angle between the vector between two atoms and the magnetic field vector.
    
    This is pure 3D trigonometry, this angle is needed to calculate coupling energies between nuclei.
    
    Parameters:
        atom1: the [x,y,z] coordinates of the first nucleus.
        atom2: the [x,y,z] coordinates of the second nucleus.
    
    Returns:
        ll: the angle between the two vectors, in degrees.
    '''
    
    zvec = [0,0,-1]
    vec = [atom1[0]-atom2[0], atom1[1]-atom2[1], atom1[2]-atom2[2]]
    a1mag = np.linalg.norm(vec)
    a2mag = np.linalg.norm(zvec)
    dotpr = np.dot(vec,zvec)
    ll = math.acos(dotpr/a1mag/a2mag)/np.pi*180
    return ll
def dist(atom1,atom2):
    '''
    Calculates the distance between two nuclei.
    
    This distance is needed to calculate coupling energies between nuclei.
    
    Parameters:
        atom1: the [x,y,z] coordinates of the first nucleus.
        atom2: the [x,y,z] coordinates of the second nucleus.
    
    Returns:
        the distance between the two nuclei, in meters.
    '''
    
    return np.sqrt(((atom1[0]-atom2[0])*10**-10)**2+((atom1[1]-atom2[1])*10**-10)**2+((atom1[2]-atom2[2])*10**-10)**2)

def dip(atom1,atom2):
    '''
    Calculates the dipolar coupling CONSTANT between two nuclei.
    
    Inputs the coordinates two nuclei, calculates their distances, then calculates the connstant,
     scaled based on the nuclei's respective gyromagnetic ratio constants.
    
    Parameters:
        atom1: the [x,y,z] coordinates of the first nucleus.
        atom2: the [x,y,z] coordinates of the second nucleus.
    
    Returns:
        vv: The dipolar coupling constant, in Hz.
    '''
    
    if atom1[3] == 113:
        gamma1 = g113
    if atom1[3] == 125:
        gamma1 = g125
    if atom2[3] == 113:
        gamma2 = g113
    if atom2[3] == 125:
        gamma2 = g125
    vv = -gamma1*gamma2*10**-7*hbar/dist(atom1,atom2)**3*2*np.pi
    if abs(vv) > 300:
        vv = -472
    return vv

In [4]:
#This iteratively generates 14,520 .txt files (in total, 24.7 MB).  
#Make sure the path on line 37 makes sense, if you're going to run it.

#load the .txt file of atomic coordinates and types
d = np.loadtxt(r"CdTe_mp-406_24.txt")
C = np.empty([12,4])
T = np.empty([12,4])
C00 = d[0]                #The central Cd is element [0]
T00 = d[13]               #The central Te is element [13]
C[0:11] = d[1:12]         #The Cd that are iteratively sampled are in elements 1-12
C[11] = d[12]             #Have to explicitly define this to avoid a bug
T[0:11] = d[14:25]        #The Te that are iteratively sampled are in elements 14-15
T[11] = d[25]             #Have to explicitly define this to avoid a bug

#Isolated Cd with 2Te3Cd
counter = 0

for Te1 in range(11):                             #Iteratively samples all Te sites for Te-125 #1
    for Te2 in range(11-Te1):                     #Iteratively samples all Te sites for Te-125 #2 (can't occupy the same site as Te-125 #1)
        for Cd1 in range(10):                     #Iteratively samples all Cd sites for Cd-113 #1
            for Cd2 in range(10-Cd1):             #Iteratively samples all Cd sites for Cd-113 #2 (can't occupy the same site as Cd-113 #1)
                for Cd3 in range(10-Cd1-Cd2):     #Iteratively samples all Cd sites for Cd-113 #3 (can't occupy the same site as Cd-113 #1 or #2)
                    counter = counter+1
                    n1 = C00                      #The central Cd-113 is always n1
                    n2 = C[Cd1]                   #The site assigned to host Cd-113 #1 is n2
                    n3 = C[Cd1+Cd2+1]             #The site assigned to host Cd-113 #2 is n3
                    n4 = C[Cd1+Cd2+Cd3+2]         #The site assigned to host Cd-113 #3 is n4
                    n5 = T00                      #The central Te-125 is always n5
                    n6 = T[Te1]                   #The site assigned to host Te-125 #1 is n6
                    n7 = T[Te1+Te2+1]             #The site assigned to host Te-125 #2 is n7
                    
                    
                    #What follows below is the generation of the input file for a specific configuration of nuclei, determined above.
                    #The .write function is used heavily
                    
                    #MAKE SURE THE PATH MAKES SENSE!:
                    f = open(r"F:\CdTeSIMPSON\PythonAutomation\SP\2Te3Cd\SP_2Te3Cd_"+str(counter)+".in","w+")   #Open new ".in" file
                    f.write("spinsys { ")
                    f.write("\n channels 113Cd 125Te")
                    f.write("\n nuclei 113Cd 113Cd 113Cd 113Cd 125Te 125Te 125Te")


                    f.write("\n jcoupling 1 5 632 0 0 0 "+str(bng(n1,n5))+" 0")                #Manually assign J-coupling to the central spin pair
                    #Calculate and write all dipolar coupling strengths between all nuclei in the configuration
                    f.write("\n dipole 1 2 "+str(dip(n1,n2))+" 0 "+str(bng(n1,n2))+" 0")
                    f.write("\n dipole 1 3 "+str(dip(n1,n3))+" 0 "+str(bng(n1,n3))+" 0")
                    f.write("\n dipole 1 4 "+str(dip(n1,n4))+" 0 "+str(bng(n1,n4))+" 0")
                    f.write("\n dipole 1 5 "+str(dip(n1,n5))+" 0 "+str(bng(n1,n5))+" 0")
                    f.write("\n dipole 1 6 "+str(dip(n1,n6))+" 0 "+str(bng(n1,n6))+" 0")
                    f.write("\n dipole 1 7 "+str(dip(n1,n7))+" 0 "+str(bng(n1,n7))+" 0")
                    f.write("\n dipole 2 3 "+str(dip(n2,n3))+" 0 "+str(bng(n2,n3))+" 0")
                    f.write("\n dipole 2 4 "+str(dip(n2,n4))+" 0 "+str(bng(n2,n4))+" 0")
                    f.write("\n dipole 2 5 "+str(dip(n2,n5))+" 0 "+str(bng(n2,n5))+" 0")
                    f.write("\n dipole 2 6 "+str(dip(n2,n6))+" 0 "+str(bng(n2,n6))+" 0")
                    f.write("\n dipole 2 7 "+str(dip(n2,n7))+" 0 "+str(bng(n2,n7))+" 0")
                    f.write("\n dipole 3 4 "+str(dip(n3,n4))+" 0 "+str(bng(n3,n4))+" 0")
                    f.write("\n dipole 3 5 "+str(dip(n3,n5))+" 0 "+str(bng(n3,n5))+" 0")
                    f.write("\n dipole 3 6 "+str(dip(n3,n6))+" 0 "+str(bng(n3,n6))+" 0")
                    f.write("\n dipole 3 7 "+str(dip(n3,n7))+" 0 "+str(bng(n3,n7))+" 0")
                    f.write("\n dipole 4 5 "+str(dip(n4,n5))+" 0 "+str(bng(n4,n5))+" 0")
                    f.write("\n dipole 4 6 "+str(dip(n4,n6))+" 0 "+str(bng(n4,n6))+" 0")
                    f.write("\n dipole 4 7 "+str(dip(n4,n7))+" 0 "+str(bng(n4,n7))+" 0")
                    f.write("\n dipole 5 6 "+str(dip(n5,n6))+" 0 "+str(bng(n5,n6))+" 0")
                    f.write("\n dipole 5 7 "+str(dip(n5,n7))+" 0 "+str(bng(n5,n7))+" 0")         
                    f.write("\n dipole 6 7 "+str(dip(n6,n7))+" 0 "+str(bng(n6,n7))+" 0")

                    #Manually assign J-coupling if any two nuclei are close enough:
                    if abs(dip(n2,n5))>300:
                        f.write("\n jcoupling 2 5 632 0 0 0 "+str(bng(n2,n5))+" 0")
                    if abs(dip(n2,n6))>300:
                        f.write("\n jcoupling 2 6 632 0 0 0 "+str(bng(n2,n6))+" 0")
                    if abs(dip(n2,n7))>300:
                        f.write("\n jcoupling 2 7 632 0 0 0 "+str(bng(n2,n7))+" 0")
                    if abs(dip(n3,n5))>300:
                        f.write("\n jcoupling 3 5 632 0 0 0 "+str(bng(n3,n5))+" 0")
                    if abs(dip(n3,n6))>300:
                        f.write("\n jcoupling 3 6 632 0 0 0 "+str(bng(n3,n6))+" 0")
                    if abs(dip(n3,n7))>300:
                        f.write("\n jcoupling 3 7 632 0 0 0 "+str(bng(n3,n7))+" 0")
                    if abs(dip(n4,n5))>300:
                        f.write("\n jcoupling 4 5 632 0 0 0 "+str(bng(n4,n5))+" 0")
                    if abs(dip(n4,n6))>300:
                        f.write("\n jcoupling 4 6 632 0 0 0 "+str(bng(n4,n6))+" 0")
                    if abs(dip(n4,n7))>300:
                        f.write("\n jcoupling 4 7 632 0 0 0 "+str(bng(n4,n7))+" 0")
                    
                    #Nitty-gritty SIMPSON code about lineshape parameters, type of experiment, etc.
                    f.write("\n}\n\n")
                    f.write("par {")
                    f.write("\n crystal_file     alpha0beta0")
                    f.write("\n variable rfCd     40000")
                    f.write("\n variable rfTe     40000")
                    f.write("\n variable tsw     10")
                    f.write("\n variable index   1")
                    f.write("\n")
                    f.write("\n sw               1e6/tsw")
                    f.write("\n np               4096")
                    f.write("\n proton_frequency 200e6")
                    f.write("\n start_operator   I5x+I6x+I7x")
                    f.write("\n detect_operator  I1p")
                    f.write("\n method           direct")
                    f.write("\n gamma_angles     1")
                    f.write("\n spin_rate        0")
                    f.write("\n verbose          1101")
                    f.write("\n}")
                    f.write("\n")
                    f.write("\nproc pulseq {} {")
                    f.write("\n global par")
                    f.write("\n")
                    f.write("\n acq_block {pulse $par(tsw) $par(rfCd) x $par(rfTe) x}")
                    f.write("\n}")
                    f.write("\n")
                    f.write("\nproc main {} {")
                    f.write("\nglobal par")
                    f.write("\n")
                    f.write("\nset f [fsimpson]")
                    f.write("\nfsave $f $par(name),$par(index).fid")
                    f.write("\n}")
                    f.close()